In [ ]:
###############################################################
##### @Title:  UWMGI generate 2.5D data
##### @Time:  2022/5/29
##### @Author: frank
##### @Describe: 
        #  part1: generate 2.5D data for train
        #  part2: generate 2.5D data for test
##### @To do: 
        #  xxxxxxxxxx
##### @Reference:
        # 
        #  UWMGI: Mask Data: https://www.kaggle.com/code/awsaf49/uwmgi-mask-data
        #  UWMGI: 2.5D stride=2 Data: https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-stride-2-data
###############################################################

In [1]:
import numpy as np
import pandas as pd
from glob import glob

# part1: Generate 2.5D data for train
 

In [2]:
#df = pd.read_csv('../input/uwmgi-mask-dataset/train.csv')
BASE_PATH = "./data"
df = pd.read_csv(f'{BASE_PATH}/mask/train.csv')
df['segmentation'] = df.segmentation.fillna('')
df['rle_len'] = df.segmentation.map(len) # length of each rle mask

#df['image_path'] = df.image_path.str.replace('/kaggle/','../') # .str: 特定列应用python字符串处理方法 
#df['mask_path'] = df.mask_path.str.replace('/kaggle/','../')

#df['mask_path'] = df.mask_path.str.replace('/png/','/np').str.replace('.png','.npy')

df['mask_path'] = df.mask_path.str.replace('/png/','/np').str.replace('.png','.npy', regex=True).str.replace("/kaggle/input/uwmgi-mask-dataset", f"{BASE_PATH}/mask")
df['image_path'] = df.image_path.str.replace("/kaggle/input/uw-madison-gi-tract-image-segmentation", f"{BASE_PATH}")

df2 = df.groupby(['id'])['segmentation'].agg(list).to_frame().reset_index() # rle list of each id
df2 = df2.merge(df.groupby(['id'])['rle_len'].agg(sum).to_frame().reset_index()) # total length of all rles of each id

df = df.drop(columns=['segmentation', 'class', 'rle_len'])
df = df.groupby(['id']).head(1).reset_index(drop=True)
df = df.merge(df2, on=['id'])
df['empty'] = (df.rle_len==0) # empty masks
display(df)

,id,case,day,slice,image_path,height,width,mask_path,segmentation,rle_len,empty
0,case123_day20_slice_0001,123,20,1,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
1,case123_day20_slice_0002,123,20,2,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
2,case123_day20_slice_0003,123,20,3,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
3,case123_day20_slice_0004,123,20,4,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
4,case123_day20_slice_0005,123,20,5,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,30,0,140,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
38492,case30_day0_slice_0141,30,0,141,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
38493,case30_day0_slice_0142,30,0,142,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True
38494,case30_day0_slice_0143,30,0,143,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True


In [3]:
##### @API: Series.shift(periods=1, freq=None, axis=0, fill_value=None)
####### periods: Number of periods to shift. Can be positive or negative.
##### @Describe: Shift index by desired number of periods with an optional time freq.

channels=5
stride=1
for i in range(channels):
    df[f'image_path_{i:02}'] = df.groupby(['case','day'])['image_path'].shift(-i*stride).fillna(method="ffill")
df['image_paths'] = df[[f'image_path_{i:02d}' for i in range(channels)]].values.tolist()
display(df)
df.image_paths[0]

,id,case,day,slice,image_path,height,width,mask_path,segmentation,rle_len,empty,image_path_00,image_path_01,image_path_02,image_path_03,image_path_04,image_paths
0,case123_day20_slice_0001,123,20,1,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,[./data/train/case123/case123_day20/scans/slic...
1,case123_day20_slice_0002,123,20,2,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,[./data/train/case123/case123_day20/scans/slic...
2,case123_day20_slice_0003,123,20,3,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,[./data/train/case123/case123_day20/scans/slic...
3,case123_day20_slice_0004,123,20,4,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,[./data/train/case123/case123_day20/scans/slic...
4,case123_day20_slice_0005,123,20,5,./data/train/case123/case123_day20/scans/slice...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,./data/train/case123/case123_day20/scans/slice...,[./data/train/case123/case123_day20/scans/slic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,30,0,140,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,[./data/train/case30/case30_day0/scans/slice_0...
38492,case30_day0_slice_0141,30,0,141,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,[./data/train/case30/case30_day0/scans/slice_0...
38493,case30_day0_slice_0142,30,0,142,./data/train/case30/case30_day0/scans/slice_01...,266,266,./data/mask/np/uw-madison-gi-tract-image-segme...,"[, , ]",0,True,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,./data/train/case30/case30_day0/scans/slice_01...,[./data/train/case30/case30_day0/scans/slice_0...
38494,case30_day0_slice_0143,30,0,143,./data/train/case30/case30_day0/scans/slic

['./data/train/case123/case123_day20/scans/slice_0001_266_266_1.50_1.50.png',
 './data/train/case123/case123_day20/scans/slice_0002_266_266_1.50_1.50.png',
 './data/train/case123/case123_day20/scans/slice_0003_266_266_1.50_1.50.png',
 './data/train/case123/case123_day20/scans/slice_0004_266_266_1.50_1.50.png',
 './data/train/case123/case123_day20/scans/slice_0005_266_266_1.50_1.50.png']

In [4]:
# numba: numpy代码并行计算
# joblib: python代码并行计算
# 目的：加速for循环

from joblib import Parallel, delayed
# joblit: running Python functions as pipeline jobs.
# Document: https://joblib.readthedocs.io/en/latest/ 

from math import sqrt

%timeit Parallel(n_jobs=1)(delayed(sqrt)(i**2) for i in range(10000))

206 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
IMAGE_DIR = './data/generation/images'
MASK_DIR = './data/generation/masks'
!mkdir -p $IMAGE_DIR && mkdir -p $MASK_DIR

IMG_SIZE = [320, 384]
ids = df['id'].unique()
print(ids)

['case123_day20_slice_0001' 'case123_day20_slice_0002'
 'case123_day20_slice_0003' ... 'case30_day0_slice_0142'
 'case30_day0_slice_0143' 'case30_day0_slice_0144']


In [6]:
import cv2
def load_img(path, size=IMG_SIZE):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    shape0 = np.array(img.shape[:2])
    resize = np.array(size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        img = np.pad(img, [pady, padx])
        #img = img.reshape((*resize))
        img = img.reshape((resize[0], resize[1]))

    return img

def load_imgs(img_paths, size=IMG_SIZE):
    imgs = np.zeros((*size, len(img_paths)), dtype=np.uint16)
    for i, img_path in enumerate(img_paths):
        img = load_img(img_path, size=size)
        imgs[..., i]+=img
    return imgs


def load_msk(path, size=IMG_SIZE):
    msk = np.load(path)
    shape0 = np.array(msk.shape[:2])
    resize = np.array(size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        msk = np.pad(msk, [pady, padx, [0,0]])
        #msk = msk.reshape((*resize, 3))
        msk = msk.reshape((resize[0], resize[1], 3))
    return msk


def save_mask(id_):
    row = df[df['id']==id_].squeeze()
    
    img_paths = row.image_paths
    imgs = load_imgs(img_paths)
    np.save(f'{IMAGE_DIR}/{id_}.npy', imgs)
    
    msk_path = row.mask_path
    msk = load_msk(msk_path)
    np.save(f'{MASK_DIR}/{id_}.npy', msk)
    
    return

In [7]:
from tqdm.notebook import tqdm
_ = Parallel(n_jobs=-1, backend='threading')(delayed(save_mask)(id_) for id_ in tqdm(ids, total=len(ids)))

  0%|          | 0/38496 [00:00<?, ?it/s]

In [ ]:
#!ls ../input/tmp/images 

In [ ]:
import matplotlib.pyplot as plt

##### @Document: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
##### Functions:
##### API: matplotlib.pyplot.figure(num=None, figsize=None, dpi=None, facecolor=None, edgecolor=None, frameon=True, FigureClass=<class 'matplotlib.figure.Figure'>, clear=False, **kwargs)[source]
##### figsize: Width, height in inches.


plt.figure(figsize=(15, 5))

imgs = np.load(f'{IMAGE_DIR}/case123_day0_slice_0080.npy').astype('float32')
imgs/=imgs.max(axis=(0,1))

##### subplot: Add an Axes to the current figure or retrieve an existing Axes.
##### subplot(nrows, ncols, index, **kwargs
##### subplot(pos, **kwargs)
plt.subplot(1, 3, 1)
##### imshow: Display data as an image, i.e., on a 2D regular raster.
plt.imshow(imgs)

msk = np.load(f'{MASK_DIR}/case123_day0_slice_0080.npy').astype('float32')
msk/=255.0

plt.subplot(1, 3, 2)
plt.imshow(msk)


plt.subplot(1, 3, 3)
plt.imshow(imgs)
plt.imshow(msk, alpha=0.5)

# 回答下如何标注的：一般医疗图像的论文一定会有医生挂名的，他们负责标数据

In [ ]:
df.to_csv('train.csv',index=False)

## 提升
New Luminide template: LB score 0.867 : https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/discussion/327166    


I guess it is not easy to achieve this "out of the box", as it is not a standard RGB image. You may try to color each of the channels with different RGB range or use different alpha values, and then somehow fuse into a regular 3-channel image.     
我想这是不容易实现的“开箱即用”，因为它不是一个标准的RGB图像。您可以尝试使用不同的RGB范围为每个通道着色，或使用不同的alpha值，然后以某种方式融合到常规的3通道图像中。      

Usually the input layer of the pre-trained networks (using imagenet) include 3 or 1 channels for RGB and grayscale images. How does changing the number of channels from 3 (as @awsaf49 did) to 5 (as you did) impacts the weights of the network? (especially in the first layer).     
通常，预训练网络的输入层（使用imagenet）包括用于RGB和灰度图像的3个或1个通道。将频道数量从3个（如@awsaf49所做的）更改为5个（如您所做的）如何影响网络的权重？（尤其是在第一层）。        

@molaee, great question! My understanding is that the pretrained weights on the first conv layer get repeated and then scaled down. For example, if the pretrained weights are for 3 channels and we try to use them with 6 channels, they get duplicated once and then multiplied by 0.5.    
我的理解是，第一个conv层上的预训练权重会重复，然后按比例缩小。例如，如果预训练权重用于3个通道，我们尝试将其用于6个通道，则它们将被复制一次，然后乘以0.5。     
I would assume that SMP automatically gets this feature from the timm library that it uses internally.
@rwightman or @pavel92 might be able to confirm.    
我假设SMP会自动从其内部使用的timm库中获取此功能。@rwightman或@pavel92可能能够确认。    

code: https://github.com/luminide/example-gi-tract   

(t-2) + t + (t+2) ==> 让t看的更远     

In [ ]:
#### TODO LIST

#### TASK1: 生成新的2.5D代码
#### 可以修改第一版代码：https://github.com/luminide/example-gi-tract/blob/2b28787cec/dataset.py  debug
#### or 可以按照上面形式预选保存生成的代码都可

#### TASK2: 利用新的2.5D代码进行训练/提交
import segmentation_models_pytorch as smp
model = smp.Unet(
            encoder_name='efficientnet-b0',
            encoder_weights=None, 
            in_channels=5,  ##### 修改in_channels即可           
            classes=3,   
            activation=None,
        )


# part2: Generate 2.5D data for test  

In [ ]:
sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
#### sub_first
sub_firset = True
sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
sub_df = sub_df[~sub_df.segmentation.isna()][:1000*3]
display(sub_df)
sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()
display(sub_df)

In [ ]:

def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row
sub_df = sub_df.apply(get_metadata, axis=1)
display(sub_df)

In [ ]:
def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    # row['id'] = f'case{case}_day{day}_slice_{slice_}'
    return row
paths = glob(f'../input/uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.apply(path2info, axis=1)
display(path_df)

In [ ]:
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')
# 2.5D MetaData
channels=3
stride=2
for i in range(channels):
    test_df[f'image_path_{i:02}'] = test_df.groupby(['case','day'])['image_path'].shift(-i*stride).fillna(method="ffill")
test_df['image_paths'] = test_df[[f'image_path_{i:02d}' for i in range(channels)]].values.tolist()
display(test_df)
